<center>

# **22AIE401 - Reinforcement Learning**  
# **Lab 3**  

</center>

### Team Members:
- Guruprasath M R - AIE22015  
- Rudraksh Mohanty - AIE22046  
- Shree Prasad M - AIE22050  
- Tharun Kaarthick G K - AIE22062  

---

### Objective:
To model a real-world warehouse navigation problem as a Markov Decision Process (MDP) and solve it using Value Iteration to find the optimal path for a robot, minimizing delivery time and avoiding obstacles. 

---

### Problem Statement:
Warehouse Robot Path Optimization using Value Iteration 
In modern warehouses (like Amazon), robots move around grid-based layouts to pick and deliver 
packages. They must: 
 - Avoid shelves (obstacles) 
 - Take the shortest and safest route
 - Deliver the package to the goal location
 - Minimize collisions and redundant moves. 


---

### Common Interpretation after completing tasks:
To be filled

## Original Code

In [ ]:
import numpy as np 
import mdptoolbox 
import matplotlib.pyplot as plt 
import random 
# Grid Parameters
rows, cols = 5, 5
num_states = rows * cols 
shelves = [(1, 1), (2, 2), (3, 3)] # Obstacle positions
actions = ['up', 'down', 'left', 'right'] 
num_actions = len(actions) 
movement = {'up': (-1, 0), 'down': (1, 0), 'left': (0, -1), 'right': (0, 1)} 
# Function to map (x, y) to index
def to_index(x, y): 
 return x * cols + y 
# Function to randomly set a goal not on a shelf
def set_dynamic_goal(): 
 possible = [(i, j) for i in range(rows) for j in range(cols) if (i, j) not
in shelves] 
 return random.choice(possible) 
# Randomly choose a goal position
goal_state = set_dynamic_goal() 
print("  Current Goal Position:", goal_state) 
# Transition and Reward Matrices
P = [np.zeros((num_states, num_states)) for _ in range(num_actions)] 
R = np.zeros((num_states, num_actions)) 
# Build MDP with stochastic transitions (intended: 0.9, unintended: 0.1 split 
across others)
for action_idx, action in enumerate(actions): 
 dx, dy = movement[action] 
 for x in range(rows): 
 for y in range(cols): 
 current_state = to_index(x, y) 
 if (x, y) == goal_state: 
 P[action_idx][current_state, current_state] = 1
 R[current_state, action_idx] = 10
 continue
 outcomes = [] 
 # Intended move (90%)
 new_x, new_y = x + dx, y + dy 
 if (new_x, new_y) in shelves or not (0 <= new_x < rows and 0 <= new_y 
< cols): 
 new_state = current_state 
 reward = -5 if (new_x, new_y) in shelves else -1
 else: 
 new_state = to_index(new_x, new_y) 
 reward = -1
 outcomes.append((new_state, 0.9, reward)) 
 # 10% misstep (wrong move in any of the other 3 directions)
 other_actions = [a for i, a in enumerate(actions) if i != action_idx] 
 for mis_action in other_actions: 
 mx, my = movement[mis_action] 
 new_x, new_y = x + mx, y + my 
 if (new_x, new_y) in shelves or not (0 <= new_x < rows and 0 <= 
new_y < cols): 
 mis_state = current_state 
 mis_reward = -5 if (new_x, new_y) in shelves else -1
 else: 
 mis_state = to_index(new_x, new_y) 
 mis_reward = -1
 outcomes.append((mis_state, 0.1 / 3, mis_reward)) 
 for s_next, prob, rew in outcomes: 
 P[action_idx][current_state, s_next] += prob 
 R[current_state, action_idx] += prob * rew # Expected reward
# Run Value Iteration
vi = mdptoolbox.mdp.ValueIteration(P, R, 0.9) 
vi.run() 
# Reshape policy to grid
policy_grid = np.array(vi.policy).reshape((rows, cols)) 
action_symbols = ['↑', '↓', '←', '→'] 
policy_symbols = np.array([[action_symbols[a] for a in row] for row in
policy_grid]) 
print("\n📍 Optimal Policy Grid:") 
print(policy_symbols) 
# Plotting
plt.figure(figsize=(6, 6)) 
for x in range(rows): 
 for y in range(cols): 
 idx = to_index(x, y) 
 if (x, y) == goal_state: 
 plt.text(y, rows - x - 1, 'G', ha='center', va='center', fontsize=14, 
color='green') 
 elif (x, y) in shelves: 
 plt.text(y, rows - x - 1, 'S', ha='center', va='center', fontsize=14, 
color='red') 
 else: 
 plt.text(y, rows - x - 1, action_symbols[vi.policy[idx]], ha='center', 
va='center', fontsize=14) 
plt.xticks(range(cols)) 
plt.yticks(range(rows)) 
plt.grid(True) 
plt.title("Stochastic Optimal Policy with Dynamic Goal") 
plt.show()